# Set Variables

In [1]:
which_chunck = 3

# -------- method: first
# which_method = "first"
which_method = "second"

if which_method == "second":
    top_k = 20


# -------- method_way
method_way = "c" # a, b, c

if (method_way == "a" and which_method == "second"):
    bug_localization_accuracy = [1] # main data - th
    knn_accuracy_threshold =    [1]

if method_way == "b":
    bug_localization_accuracy = [1] # main data - th
    knn_accuracy_threshold = [
        0.5, 0.55, 
        0.6, 0.65, 
        0.7, 0.75, 
        0.8, 0.85, 
        0.9, 0.95
    ]


if method_way == "c":
    bug_localization_accuracy = [0.8]
    knn_accuracy_threshold = [
        0.5, 0.55, 
        0.6, 0.65, 
        0.7, 0.75, 
        0.8, 0.85, 
        0.9, 0.95
    ]


# -------- dataset
# software_name = "camel"
# software_name = "cloudstack"
software_name = "geode"
# software_name = "HBase"


# -------- google colab
# on_google_colab = True
on_google_colab = False

In [2]:
print("bla: ", bug_localization_accuracy)
print("th:  ", knn_accuracy_threshold)

bla:  [0.8]
th:   [0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]


# Google Colab

In [3]:
if software_name == "camel":
    dataset_file_name = "dataset_camel - v.01"
    dataset_file_name_ts = "timeserAll_camel - v.01"
    
elif software_name == "cloudstack":
    dataset_file_name = "cloudstack_bugs - v.01"
    dataset_file_name_ts = "cloudstack_TimeSeries - v.01"
    
elif software_name == "geode":
    dataset_file_name = "geode_Bug - v.01"
    dataset_file_name_ts = "geode_TS - v.01"
    
else:
    dataset_file_name = "dataset_hbase - v.03"
    dataset_file_name_ts = "timeserAll - v.03"

In [4]:
if on_google_colab:
    
    !pip install enlighten
    !pip install --upgrade statsmodels 
    !pip install --upgrade matplotlib
    
    # load data from google drive
    from google.colab import drive
    drive.mount("/content/gdrive", force_remount=True)
    !ls "/content/gdrive/My Drive/"
    
    tempPre = "gdrive/MyDrive/Colab Notebooks/paper/Indirect Method/"
    pre_path_data = tempPre + "data/" + software_name + "/"
    
    # you can choose one of these and comment out the other:
    # pre_path_output = tempPre + "output/"
    pre_path_output = tempPre + "output/" + dataset_file_name + "/" + which_method + "/"

else:
    tempPre =    "E:/darsy/00/02- arshad/10- paper code/"
    tempData =   "00- My Data/Datasets/Indirect Method/"
    tempOutput = "01- Jupyter Notebook/InDirect Method/00. Output/"
    pre_path_data =   tempPre + tempData   + software_name + "/"
    pre_path_output = tempPre + tempOutput + software_name + "/" + dataset_file_name + "/" + which_method + "/"

# Libs

In [5]:
import enlighten
import pandas as pd
import numpy as np

from statsmodels.tsa.stattools import adfuller
from sklearn.neighbors import KNeighborsClassifier

In [6]:
import warnings
warnings.filterwarnings("ignore")

# Config

In [7]:
if which_method == "first":
    path_folder = pre_path_data + "first/bla={}/".format(bug_localization_accuracy[0])
    
    sub_chunck_file_org =  "{} _ bla={} - {}.csv".format(dataset_file_name, 
                                                         bug_localization_accuracy[0], which_chunck)
    
    sub_chunck_file_pred = "{} _ bla={} - {}-pred.csv".format(dataset_file_name, 
                                                              bug_localization_accuracy[0], which_chunck)
    
else:
    path_folder = pre_path_data + "second/k={}/bla={}/".format(top_k, bug_localization_accuracy[0])
    
    sub_chunck_file_org =  "{} _ k={}, bla={} - {}.csv".format(dataset_file_name, top_k, 
                                                               bug_localization_accuracy[0], which_chunck)
    
    sub_chunck_file_pred = "{} _ k={}, bla={} - {}-pred.csv".format(dataset_file_name, top_k, 
                                                                    bug_localization_accuracy[0], which_chunck)



mypaths = {
    "split_dataset":      path_folder   + sub_chunck_file_org,
    "timeser":            pre_path_data + dataset_file_name_ts + ".csv",
    "split_dataset_pred": path_folder   + "pred/" + sub_chunck_file_pred
    
}


preprocessing_params = {
    "dataset": {
        # time_stamp is the time that we change the class for a bug fixing.
        # id stands for identifier
        "columns_name":   ["bug_report_id", "class_id", "time_stamp", "label"],
        "columns_dtype" : {0: "int64", 1: "int64", 2: "int64", 3:"int32"}
    },
    
    "timeser": {
        "columns_name":   ["class_id", "time_stamp", "label"],
        "columns_dtype" : {0: "int64", 1: "int64", 2:"int32"}
    }
}
    

knn_model_params = {
    "train_size": 0.8,
    "n_neighbors": 5
}

progress_colours = ["webmaroon", "salmon", "orangered", "deeppink3", "crimson", 
                    "black", "blue", "gray", "cyan4", "darkgreen"]

# Read Files

## df_timeserAll

In [8]:
df_timeserAll = pd.read_csv(
    mypaths["timeser"], 
    names=preprocessing_params["timeser"]["columns_name"], 
    dtype=preprocessing_params["timeser"]["columns_dtype"], 
    header=None
)

## df_data

In [9]:
print(mypaths["split_dataset"])

df_data = pd.read_csv(
    mypaths["split_dataset"], 
    names=preprocessing_params["dataset"]["columns_name"], 
    dtype=preprocessing_params["dataset"]["columns_dtype"], 
    header=None
)

df_data = [df_data]
df_data

E:/darsy/00/02- arshad/10- paper code/00- My Data/Datasets/Indirect Method/geode/second/k=20/bla=0.8/geode_Bug - v.01 _ k=20, bla=0.8 - 3.csv


[      bug_report_id  class_id  time_stamp  label
 0             34007     82645  1489106000      0
 1             34007     91501  1489106000      0
 2             34007    103455  1489106000      0
 3             34007    103500  1489106000     -1
 4             34007    104356  1489106000      0
 ...             ...       ...         ...    ...
 7007          39606    113784  1435703450      0
 7008          39622    114566  1434766441      0
 7009          39622    114589  1434766441      0
 7010          39622    114541  1435080450      0
 7011          39623    114864  1434656000      0
 
 [7012 rows x 4 columns]]

# MyTimeSer

In [10]:
class MyTimeSer():
    
    def __init__(self, df_timeserAll, df_data, knn_model_params, threshold, bug_localization_accuracy):
        self.df_data_pred = df_data
        self.df_timeserAll = df_timeserAll
        self.accepted_num = {}
        self.threshold = threshold # [0.5, 0.5, ...]
        self.bla = bug_localization_accuracy
        self.knn_model_params = knn_model_params
        self.time_to_train = None
        
        
        for i, bla in enumerate(self.bla):
            self.accepted_num[bla] = {"isctss": 0}
            temp_df_data_pred = self.df_data_pred[i]
            
            for th in threshold:
                # add prediction columns
                new_column_name = self.prediction_column_name(th)
                temp_df_data_pred[new_column_name] = np.nan
                
                # initialize self.accepted_num
                # self.accepted_num[bla][th]: (iscts &) accup
                self.accepted_num[bla][th] = 0
    
    
    # ************************** helper ************************** #
    
    def prediction_column_name(self, th):
        return "predictions_" + str(th)
    
    
    # ************************** progress lines ************************** #
    
    def progress_lines(self, colours):
        desc = self.set_strings_to_equal_len_()
        total = len(self.df_data_pred[0])
        manager = enlighten.get_manager()
        progresses = []
        for i, bla in enumerate(self.bla):
            prog = manager.counter(total=total, desc=desc[i], unit="sample", color=colours[i])
            prog.refresh()
            progresses.append(prog)
        self.progresses = progresses
    
    
    def set_strings_to_equal_len_(self):
        words = []
        for bla in self.bla:
            desc = "train-test-" + str(bla)
            words.append(desc)
        
        max_len = 0
        longest_string_length = len(max(words, key=len))
        w = []
        for i, word in enumerate(words):
            temp = longest_string_length - len(word)
            w.append(word + " " * temp)
        return w
    
    
    # ************************** prediction ************************** #
    
    def make_predictions(self):
        for i, bla in enumerate(self.bla):
            # --- make prediction
            self.df_data_pred[i].apply(lambda row: self.flow_(row, i, bla), axis=1)

        # --- time to train
        totalSeconds = self.progresses[-1].elapsed
        hours = int(totalSeconds / 3600)
        minutes = int((totalSeconds - (hours * 3600)) / 60)
        seconds = int((totalSeconds - (hours * 3600 + minutes * 60)))
        self.time_to_train = f"{hours}h:{minutes}m:{seconds}s"
    
    
    def flow_(self, row, i, bla):
        # --- extract class time series.
        cts = self.extract_class_time_series_(row["class_id"], row["time_stamp"])
        
        # --- if cts is stationary, then make prediction & if accuracy pass the threshold, save the prediction
        is_cts_stationary = self.testSeries_(cts)
        if is_cts_stationary:
            self.accepted_num[bla]["isctss"] += 1
            accuracy, yhat = self.predictValue_(cts)
            for th in self.threshold:
                if accuracy > th:
                    self.accepted_num[bla][th] += 1
                    tempColumnName = self.prediction_column_name(th)
                    self.df_data_pred[i].loc[row.name, tempColumnName] = int(yhat)
        
        self.progresses[i].update()
    
    
    def extract_class_time_series_(self, class_id, bug_class_commitTime):
        df_class_time_series = self.df_timeserAll[
            (self.df_timeserAll["class_id"]  == class_id) & 
            (self.df_timeserAll["time_stamp"] < bug_class_commitTime)
        ]
        sorted_df_cts = df_class_time_series.sort_values(by="time_stamp", ascending=True)
        cts = sorted_df_cts["label"].to_numpy()
        return cts
    
    
    def testSeries_(self, cts):
        if len(cts) <= 10:
            return False
        
        result = adfuller(cts)
        critical_values = result[4].items()
        is_cts_stationary = True
        
        for alpha, critical_value in critical_values:
            ADF_Statistic = result[0]
            if critical_value < ADF_Statistic:
                is_cts_stationary = False
                break
        return is_cts_stationary
    
    
    def predictValue_(self, cts):
        max_accuracy = -1
        best_yhat = -1
        for query_length in range(3, 10):
            knnX, knnY = self.knnDS_(cts, query_length)
            query = np.array(cts[-query_length:])
            accuracy, yhat = self.knnPredict_(knnX, knnY, query)
            if accuracy > max_accuracy:
                max_accuracy = accuracy
                best_yhat = yhat
        return (max_accuracy, best_yhat)
    
    
    def knnDS_(self, cts, ql):
        knnX = [cts[i: i + ql] for i in range(len(cts) - ql)] # [[0, -1, 1, 1, ...], [1, 0, 0, 0, ...]]
        knnY = [cts[i + ql]    for i in range(len(cts) - ql)] # [0, 1, ...]
        return knnX, knnY
    
    
    # x: [[0, -1, 1, 1, ...], [1, 0, 0, 0, ...]]
    # y: [0, 1, ...]
    def knnPredict_(self, x, y, query):
        # --- prepare train and test data
        size = int(len(x) * self.knn_model_params["train_size"])
        xtrain, xtest = x[0: size], x[size:]
        ytrain, ytest = y[0: size], y[size:]
        
        if len(xtrain) < 6:
            return 0, 0
        
        # --- train model
        knn = KNeighborsClassifier(n_neighbors=self.knn_model_params["n_neighbors"])
        knn.fit(xtrain, ytrain)
        
        # --- test the model
        accuracy = knn.score(xtest, ytest)
        
        # --- make prediction
        yhat = 0
        query = query.reshape(1, -1)
        yhat = knn.predict(query)
        return accuracy, yhat
    
    
    def free_memory(self):
        self.df_timeserAll = None

## obj

In [11]:
mt_data = MyTimeSer(
    df_timeserAll, 
    df_data, 
    knn_model_params, 
    knn_accuracy_threshold, 
    bug_localization_accuracy)

In [12]:
if knn_accuracy_threshold[0] == 1:
    eee = df_data[0].copy()
    eee["predictions_1"] = eee["label"].copy()
    mt_data.df_data_pred = [eee]
    mt_data.free_memory()
else:
    print("else")
    mt_data.progress_lines(progress_colours)
    mt_data.make_predictions()

else


In [13]:
mt_data.free_memory()

print(mt_data.time_to_train)
print(len(mt_data.df_data_pred))

mt_data.df_data_pred[0]

0h:2m:30s
1


,bug_report_id,class_id,time_stamp,label,predictions_0.5,predictions_0.55,predictions_0.6,predictions_0.65,predictions_0.7,predictions_0.75,predictions_0.8,predictions_0.85,predictions_0.9,predictions_0.95
0,34007,82645,1489106000,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,34007,91501,1489106000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,34007,103455,1489106000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,34007,103500,1489106000,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,34007,104356,1489106000,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7007,39606,113784,1435703450,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7008,39622,114566,1434766441,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7009,39622,114589,1434766441,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7010,39622,114541,1435080450,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# save

In [14]:
df_data[0].to_csv(mypaths["split_dataset_pred"], index=False, na_rep="NULL")